# Homework 6: Mixed effects

This homework assignment is designed to give you practice fitting and interpreting mixed effects models. 

We will be using the **LexicalData.csv** and **Items.csv** files from the *Homework/lexDat* folder in the class GitHub repository again. 

This data is a subset of the [English Lexicon Project database](https://elexicon.wustl.edu/). It provides the reaction times (in milliseconds) of many subjects as they are presented with letter strings and asked to decide, as quickly and as accurately as possible, whether the letter string is a word or not. The **Items.csv** provides characteristics of the words used, namely frequency (how common is this word?) and length (how many letters?). Unlike in the previous homework, there isn't any missing data in the **LexicalData.csv** file. 

*Data courtesy of Balota, D.A., Yap, M.J., Cortese, M.J., Hutchison, K.A., Kessler, B., Loftis, B., Neely, J.H., Nelson, D.L., Simpson, G.B., & Treiman, R. (2007). The English Lexicon Project. Behavior Research Methods, 39, 445-459.*

---
## 1. Loading and formatting the data (1 point)

Load in data from the **LexicalData.csv** and **Items.csv** files. As in the previous homeworks, remove the commas from the reaction times and convert them from strings to numbers. Use `left_join` to add word characteristics `Length` and `Log_Freq_Hal` from **Items** to **LexicalData**. 

*Note: the `Freq_HAL` variable in **Items.csv** has a similar formatting issue, using string values with commas. We're not going to worry about fixing this since we're only using `Log_Freq_HAL`, which is the natural log transformation of `Freq_HAL`, in this homework.*

In [1]:
install.packages('tidyverse')
install.packages('here')
install.packages('skimr')
install.packages('janitor')
install.packages('psych')
install.packages('reshape2')
install.packages('dplyr')






library(tidyverse)
library(here)
library(skimr)
library(janitor)
library(psych)
library(reshape2)
library(dplyr)

setwd("~/Downloads/Luetkehans_DSPN_S22/data")

lexdata <- read.csv("LexicalData.csv")
items <- read.csv("Items.csv")

head(lexdata)
head(items)



The downloaded binary packages are in
	/var/folders/1z/t3c0jklx1v70qk4jxjj4kx_h0000gn/T//RtmpHpT1g2/downloaded_packages

The downloaded binary packages are in
	/var/folders/1z/t3c0jklx1v70qk4jxjj4kx_h0000gn/T//RtmpHpT1g2/downloaded_packages

The downloaded binary packages are in
	/var/folders/1z/t3c0jklx1v70qk4jxjj4kx_h0000gn/T//RtmpHpT1g2/downloaded_packages

The downloaded binary packages are in
	/var/folders/1z/t3c0jklx1v70qk4jxjj4kx_h0000gn/T//RtmpHpT1g2/downloaded_packages

The downloaded binary packages are in
	/var/folders/1z/t3c0jklx1v70qk4jxjj4kx_h0000gn/T//RtmpHpT1g2/downloaded_packages

The downloaded binary packages are in
	/var/folders/1z/t3c0jklx1v70qk4jxjj4kx_h0000gn/T//RtmpHpT1g2/downloaded_packages

The downloaded binary packages are in
	/var/folders/1z/t3c0jklx1v70qk4jxjj4kx_h0000gn/T//RtmpHpT1g2/downloaded_packages


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

here() starts at /Users/SidMotwani/Downloads


Attaching package: ‘janitor’


The following objects are masked from ‘package:stats’:

    chisq.test, fisher.test



Attaching package: ‘psych’


The following objects are masked from ‘package:ggplot2’:

    %+%, alpha



Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths




,Sub_ID,Trial,Type,D_RT,D_Word,Outlier,D_Zscore
,<int>,<int>,<int>,<chr>,<chr>,<chr>,<dbl>
1,157,1,1,710,browse,false,-0.437
2,67,1,1,"1,094",refrigerant,false,0.825
3,120,1,1,587,gaining,false,-0.645
4,21,1,1,984,cheerless,false,0.025
5,236,1,1,577,pattered,false,-0.763
6,236,2,1,715,conjures,false,-0.364


,Occurrences,Word,Length,Freq_HAL,Log_Freq_HAL
,<int>,<chr>,<int>,<chr>,<dbl>
1,1,synergistic,11,284,5.649
2,1,synonymous,10,951,6.858
3,1,syntactical,11,114,4.736
4,1,synthesis,9,"6,742",8.816
5,1,synthesized,11,"2,709",7.904
6,1,synthesizer,11,"1,390",7.237


In [3]:
#setwd("/Users/josephluetkehans/Documents/GitHub/Luetkehans_DSPN_S22/data")

lexdata <- read.csv("LexicalData.csv")
items <- read.csv("Items.csv")

head(lexdata)
head(items)

,Sub_ID,Trial,Type,D_RT,D_Word,Outlier,D_Zscore
,<int>,<int>,<int>,<chr>,<chr>,<chr>,<dbl>
1,157,1,1,710,browse,false,-0.437
2,67,1,1,"1,094",refrigerant,false,0.825
3,120,1,1,587,gaining,false,-0.645
4,21,1,1,984,cheerless,false,0.025
5,236,1,1,577,pattered,false,-0.763
6,236,2,1,715,conjures,false,-0.364


,Occurrences,Word,Length,Freq_HAL,Log_Freq_HAL
,<int>,<chr>,<int>,<chr>,<dbl>
1,1,synergistic,11,284,5.649
2,1,synonymous,10,951,6.858
3,1,syntactical,11,114,4.736
4,1,synthesis,9,"6,742",8.816
5,1,synthesized,11,"2,709",7.904
6,1,synthesizer,11,"1,390",7.237


In [4]:
lexdata <- lexdata %>% 
mutate(D_RT_clean = as.numeric(gsub(",", "", as.character(D_RT)))) %>%
na.omit() %>%
subset(select = -c(D_RT))

head(lexdata)

colnames(lexdata) <- c('Sub_ID', 'Trial', 'Type', 'Word', 'Outlier', 'D_Zscore', 'D_RT')

lexdata <- left_join(lexdata, items, by = "Word") #%>%
#subset(select = -c(Occurrences, Freq_HAL))

#For some reason, trying to remove the columns that we don't need breaks the code?  
#Does it matter if I still have occurrences and Freq_HAL?  

head(lexdata)


,Sub_ID,Trial,Type,D_Word,Outlier,D_Zscore,D_RT_clean
,<int>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>
1,157,1,1,browse,false,-0.437,710
2,67,1,1,refrigerant,false,0.825,1094
3,120,1,1,gaining,false,-0.645,587
4,21,1,1,cheerless,false,0.025,984
5,236,1,1,pattered,false,-0.763,577
6,236,2,1,conjures,false,-0.364,715


,Sub_ID,Trial,Type,Word,Outlier,D_Zscore,D_RT,Occurrences,Length,Freq_HAL,Log_Freq_HAL
,<int>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>,<int>,<int>,<chr>,<dbl>
1,157,1,1,browse,false,-0.437,710,2,6,"7,016",8.856
2,67,1,1,refrigerant,false,0.825,1094,3,11,104,4.644
3,120,1,1,gaining,false,-0.645,587,4,7,"4,039",8.304
4,21,1,1,cheerless,false,0.025,984,4,9,14,2.639
5,236,1,1,pattered,false,-0.763,577,4,8,4,1.386
6,236,2,1,conjures,false,-0.364,715,4,8,194,5.268


---
## 2. Model fitting (4 points)

First, fit a linear model with `Log_Freq_HAL` and `Length` as predictors, and `D_RT` as the output. Include an interaction term. Use `summary()` to look at the model output. 

In [5]:
lexlinearmodel <- lm(D_RT ~ Log_Freq_HAL + Length + Log_Freq_HAL:Length, data = lexdata)

summary(lexlinearmodel)




Call:
lm(formula = D_RT ~ Log_Freq_HAL + Length + Log_Freq_HAL:Length, 
    data = lexdata)

Residuals:
     Min       1Q   Median       3Q      Max 
-1118.01  -205.23   -86.95    90.77  3147.07 

Coefficients:
                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)         610.1903    14.6678  41.601  < 2e-16 ***
Log_Freq_HAL         -6.0239     1.9678  -3.061  0.00221 ** 
Length               47.7531     1.6368  29.175  < 2e-16 ***
Log_Freq_HAL:Length  -2.9421     0.2348 -12.528  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 359.1 on 62606 degrees of freedom
Multiple R-squared:  0.09473,	Adjusted R-squared:  0.09469 
F-statistic:  2184 on 3 and 62606 DF,  p-value: < 2.2e-16


Call:
lm(formula = D_RT ~ Log_Freq_HAL + Length + Log_Freq_HAL:Length, 
    data = lexdata)

Residuals:
     Min       1Q   Median       3Q      Max 
-1118.01  -205.23   -86.95    90.77  3147.07 

Coefficients:
                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)         610.1903    14.6678  41.601  < 2e-16 ***

Log_Freq_HAL         -6.0239     1.9678  -3.061  0.00221 ** 

Length               47.7531     1.6368  29.175  < 2e-16 ***

Log_Freq_HAL:Length  -2.9421     0.2348 -12.528  < 2e-16 ***

---
Signif. codes:  
0 ‘***’ 
0.001 ‘**’ 
0.01 ‘*’ 
0.05 ‘.’ 
0.1 ‘ ’ 

Residual standard error: 

359.1 on 62606 degrees of freedom

Multiple R-squared:  0.09473,	
Adjusted R-squared:  0.09469 
F-statistic:  2184 on 3 and 62606 DF,  
p-value: < 2.2e-16

Now, install `lme4` using `install.packages()` and then load the library. 

In [6]:
install.packages('lme4')
library(lme4)

#Why won't this install???



The downloaded binary packages are in
	/var/folders/1z/t3c0jklx1v70qk4jxjj4kx_h0000gn/T//RtmpHpT1g2/downloaded_packages


Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack




> note: I wasn't able to install the lme4 package on my own computer, so I ran it on a friend's laptop.  I think the output for the summary of the lmer() model remains regardless of whether I had the package installed when I ran the code, but I figured I'd mention it.  Thank you!

Now fit a mixed effects model that includes the same predictors as the linear model above, as well as random intercepts for `Sub_ID` (i.e., cases where subject ID shifts the RT mean). Use `summary()` to look at the model output. 

In [7]:
lexlinearmodel_mixed <- lmer(D_RT ~ Log_Freq_HAL + Length + Log_Freq_HAL:Length + (1|Sub_ID), data = lexdata)

summary(lexlinearmodel_mixed)



Linear mixed model fit by REML ['lmerMod']
Formula: D_RT ~ Log_Freq_HAL + Length + Log_Freq_HAL:Length + (1 | Sub_ID)
   Data: lexdata

REML criterion at convergence: 888235.6

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.5058 -0.5472 -0.1568  0.3103 10.7381 

Random effects:
 Groups   Name        Variance Std.Dev.
 Sub_ID   (Intercept) 46333    215.3   
 Residual             82978    288.1   
Number of obs: 62610, groups:  Sub_ID, 299

Fixed effects:
                    Estimate Std. Error t value
(Intercept)         616.8445    17.1522  35.963
Log_Freq_HAL         -7.4374     1.5830  -4.698
Length               47.7477     1.3162  36.277
Log_Freq_HAL:Length  -2.8778     0.1888 -15.239

Correlation of Fixed Effects:
            (Intr) Lg_F_HAL Length
Log_Frq_HAL -0.645                
Length      -0.656  0.917         
Lg_Fr_HAL:L  0.582 -0.942   -0.923

Linear mixed model fit by REML ['lmerMod']
Formula: D_RT ~ Log_Freq_HAL + Length + Log_Freq_HAL:Length + (1 | Sub_ID)
   Data: lexdata

REML criterion at convergence: 888235.6

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.5058 -0.5472 -0.1568  0.3103 10.7381 

Random effects:
 Groups   Name        Variance Std.Dev.
 Sub_ID   (Intercept) 46333    215.3   
 Residual             82978    288.1   
Number of obs: 62610, groups:  Sub_ID, 299

Fixed effects:
                    Estimate Std. Error t value
(Intercept)         616.8445    17.1522  35.963
Log_Freq_HAL         -7.4374     1.5830  -4.698
Length               47.7477     1.3162  36.277
Log_Freq_HAL:Length  -2.8778     0.1888 -15.239

Correlation of Fixed Effects:
            (Intr) Lg_F_HAL Length
Log_Frq_HAL -0.645                
Length      -0.656  0.917         
Lg_Fr_HAL:L  0.582 -0.942   -0.923

---
## 3. Model assessment (4 points)

Compare the three t-values for the fixed effects and the mixed effects models. How do they differ, and why? 

> t-values for fixed effects model: 
        log_freq_HAL: -3.061
              Length:  29.175
log_freq_HAL* Length: -12.528

> t-values for mixed effects model:
        log_freq_HAL: -4.698
              Length:  36.277
log_freq_HAL* Length: -15.239

> The t-values for the mixed effects model are further from 0 than any of those for the fixed effects model.  I believe that this means that the standard errors of the mixed effects model are smaller relative to the coefficients of that model than they are for the fixed effects model. I also believe that this means the mixed effects model suggests a stronger effect of each of the variables than the fixed effect model.

Use the Aikeke Information Criterion (AIC) to compare these two models. Which one is better? 

In [8]:
aic = AIC(lexlinearmodel, lexlinearmodel_mixed)
aic



,df,AIC
,<dbl>,<dbl>
lexlinearmodel,5,914436.4
lexlinearmodel_mixed,6,888247.6


> According to the AIC comparison, the mixed effects model is a better fit for this data because it has a lower AIC.  

---
##  4. Reflection (1 point)

What other random effects could be controlled for in this data set? 

> Trial could be controlled for in this data set as a random effect.   

**DUE:** 5pm EST, March 25, 2022

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here. 
> *Someone's Name*